In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

In [103]:
dfh18 = pd.read_csv("500_Cities__Local_Data_for_Better_Health__2018_release.csv")

In [104]:
#Display dataframe
display(dfh18.head())
display(dfh18.info())

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2016,US,United States,NaN,US,BRFSS,Prevention,59,Current lack of health insurance among adults ...,%,...,14.9,NaN,NaN,308745538,NaN,PREVENT,ACCESS2,NaN,NaN,Health Insurance
1,2016,US,United States,NaN,US,BRFSS,Prevention,59,Current lack of health insurance among adults ...,%,...,11.8,NaN,NaN,308745538,NaN,PREVENT,ACCESS2,NaN,NaN,Health Insurance
2,2016,US,United States,NaN,US,BRFSS,Health Outcomes,59,Arthritis among adults aged >=18 Years,%,...,23.2,NaN,NaN,308745538,NaN,HLTHOUT,ARTHRITIS,NaN,NaN,Arthritis
3,2016,US,United States,NaN,US,BRFSS,Health Outcomes,59,Arthritis among adults aged >=18 Years,%,...,25.6,NaN,NaN,308745538,NaN,HLTHOUT,ARTHRITIS,NaN,NaN,Arthritis
4,2016,US,United States,NaN,US,BRFSS,Unhealthy Behaviors,59,Binge drinking among adults aged >=18 Years,%,...,18.2,NaN,NaN,308745538,NaN,UNHBEH,BINGE,NaN,NaN,Binge Drinking


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810103 entries, 0 to 810102
Data columns (total 24 columns):
Year                          810103 non-null int64
StateAbbr                     810103 non-null object
StateDesc                     810103 non-null object
CityName                      810047 non-null object
GeographicLevel               810103 non-null object
DataSource                    810103 non-null object
Category                      810103 non-null object
UniqueID                      810103 non-null object
Measure                       810103 non-null object
Data_Value_Unit               810103 non-null object
DataValueTypeID               810103 non-null object
Data_Value_Type               810103 non-null object
Data_Value                    787311 non-null float64
Low_Confidence_Limit          787311 non-null float64
High_Confidence_Limit         787311 non-null float64
Data_Value_Footnote_Symbol    22794 non-null object
Data_Value_Footnote           22794 non-

None

In [105]:
#Filter for desired columns, remove redundant columns and rename columns
dfh18 = dfh18[['Year', 'GeographicLevel', 'StateDesc', 'CityName','Short_Question_Text','Measure','Data_Value_Type', 
               'Data_Value','Low_Confidence_Limit', 'High_Confidence_Limit','PopulationCount','GeoLocation',
               'CityFIPS','TractFIPS']]
dfh18.columns = ["Year","GeographicLevel","State","City","Category","Measure","Data_Type",
                  "Data_Value","Low_Confidence_Limit","High_Confidence_Limit","Population","Tract GeoLocation",
                  "CityFIPS","TractFIPS"]

In [106]:
# Filter out rows at the US or City level
dfh18 = dfh18.loc[dfh18["GeographicLevel"] == "Census Tract"]

In [107]:
#Filter values related to heart disease
dfh18 = dfh18.loc[dfh18["Measure"].isin(['High blood pressure among adults aged >=18 Years',
                                         'Coronary heart disease among adults aged >=18 Years',
                                         'High cholesterol among adults aged >=18 Years who have been screened in the past 5 Years',
                                         'Current smoking among adults aged >=18 Years',
                                         'Diagnosed diabetes among adults aged >=18 Years',
                                         'No leisure-time physical activity among adults aged >=18 Years',
                                         'Obesity among adults aged >=18 Years'])]

In [108]:
# Remove rows with no data value
dfh18 = dfh18.loc[dfh18.Data_Value.notnull()]

In [109]:
# Store values in separate variables
smoking = dfh18.loc[dfh18["Category"] == "Current Smoking"]
hypertension = dfh18.loc[dfh18["Category"] == "High Blood Pressure"]
obesity = dfh18.loc[dfh18["Category"] == "Obesity"]
sedentery = dfh18.loc[dfh18["Category"] == "Physical Inactivity"]
cholesterol = dfh18.loc[dfh18["Category"] == "High Cholesterol"]
diabetes = dfh18.loc[dfh18["Category"] == "Diabetes"]
heartd = dfh18.loc[dfh18["Category"] == "Coronary Heart Disease"]

In [110]:
# Select the population and tract columns for future merging
tractpop = dfh18[["Population","TractFIPS"]]
tractpop.head()

,Population,TractFIPS
283,3257,1.073002e+09
305,3629,1.073002e+09
361,3042,1.073000e+09
362,2735,1.073000e+09
363,3338,1.073000e+09


In [111]:
# Read zip-to-tract conversion file
tractzip = pd.read_excel("Tract-zip.xlsx")

# Extract desired columns and display the df
tractzip = tractzip[["zip","tract","res_ratio"]]
display(tractzip.head())

display(tractzip.info())

,zip,tract,res_ratio
0,36067,1001020100,1.000000
1,36067,1001020200,1.000000
2,36067,1001020300,1.000000
3,36067,1001020400,0.025014
4,36066,1001020400,0.974986


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171501 entries, 0 to 171500
Data columns (total 3 columns):
zip          171501 non-null int64
tract        171501 non-null int64
res_ratio    171501 non-null float64
dtypes: float64(1), int64(2)
memory usage: 3.9 MB


None

In [112]:
# Merge the conversion file with the health df's population column
tractzippop = pd.merge(tractzip,tractpop, how="inner", left_on="tract", right_on="TractFIPS")
tractzippop.drop(columns="TractFIPS",inplace=True)
tractzippop = tractzippop.sort_values("zip")
tractzippop.head()

,zip,tract,res_ratio,Population
168952,1013,25013800400,0.015906,6726
168936,1013,25013800300,0.001006,4395
168937,1013,25013800300,0.001006,4395
168933,1013,25013800300,0.001006,4395
168932,1013,25013800300,0.001006,4395


In [113]:
# Group by zip code and multiply the population rows by the resident ratio rows
# resulting in a population for each zip code
popzipgroups = tractzippop.groupby("zip").apply(lambda x: np.sum(x["res_ratio"] * x["Population"])).reset_index()
popzipgroups.columns = ["Zip Code","Population"]
popzipgroups = popzipgroups.sort_values("Zip Code")
popzipgroups.head()

,Zip Code,Population
0,1013,779.848151
1,1020,218.076681
2,1028,284.540617
3,1095,31.833239
4,1101,9138.149918


In [114]:
# Filter out rows with a population of 0
popzipgroups = popzipgroups.loc[popzipgroups.Population != 0]

popzipgroups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7522 entries, 0 to 8639
Data columns (total 2 columns):
Zip Code      7522 non-null int64
Population    7522 non-null float64
dtypes: float64(1), int64(1)
memory usage: 176.3 KB


In [115]:
# Display rows with tracts that have 50% of residents in one zip code
halves = tractzip[tractzip.res_ratio == .5]
halves.sort_values("tract")
display(halves.head())

# Arbitratily drop one zip code from tracts with a 50/50 split
tractzip = tractzip.drop([3550,10175,12347,18703,19308,24757,35675,68346,87135,145657,170353])

,zip,tract,res_ratio
3549,99825,2105000300,0.5
3550,99826,2105000300,0.5
10174,93611,6019005701,0.5
10175,93612,6019005701,0.5
12345,90077,6037265301,0.5


In [116]:
# Group by tracts and select rows with the greatest resident percentage
tractgroups = tractzip.groupby("tract")["res_ratio"].max().reset_index()
display(tractgroups.head())

# Merge tract groups with the original df containing zip codes
# resulting in one zip code per tract
tractzipm = pd.merge(tractzip,tractgroups, how="inner")
tractzipm.columns = ["Zip Code","tract","res_ratio"]
display(tractzipm.head())

display(tractzipm.info())

,tract,res_ratio
0,1001020100,1.000000
1,1001020200,1.000000
2,1001020300,1.000000
3,1001020400,0.974986
4,1001020500,0.798168


,Zip Code,tract,res_ratio
0,36067,1001020100,1.000000
1,36067,1001020200,1.000000
2,36067,1001020300,1.000000
3,36066,1001020400,0.974986
4,36066,1001020500,0.798168


<class 'pandas.core.frame.DataFrame'>
Int64Index: 73648 entries, 0 to 73647
Data columns (total 3 columns):
Zip Code     73648 non-null int64
tract        73648 non-null int64
res_ratio    73648 non-null float64
dtypes: float64(1), int64(2)
memory usage: 2.2 MB


None

In [117]:
# Merge health values with conversion dataframe
smokingm = pd.merge(smoking,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
hypertensionm = pd.merge(hypertension,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
obesitym = pd.merge(obesity,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
sedenterym = pd.merge(sedentery,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
cholesterolm = pd.merge(cholesterol,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
diabetesm = pd.merge(diabetes,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
heartdm = pd.merge(heartd,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")

In [118]:
# Group by zip code for each value and calculate weighted mean prevalence
smokingmp = smokingm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
smokingmp.columns = ["Zip Code","Smoking Prevalence"]

hypertensionmp = hypertensionm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
hypertensionmp.columns = ["Zip Code","Hypertension Prevalence"]

obesitymp = obesitym.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
obesitymp.columns = ["Zip Code","Obesity Prevalence"]

sedenterymp = sedenterym.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
sedenterymp.columns = ["Zip Code","Sedentarism Prevalence"]

cholesterolmp = cholesterolm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
cholesterolmp.columns = ["Zip Code","Cholesterol Prevalence"]

diabetesmp = diabetesm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
diabetesmp.columns = ["Zip Code","Diabetes Prevalence"]

heartdmp = heartdm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
heartdmp.columns = ["Zip Code","Heart Disease Prevalence"]

#Merge the values
_ = pd.merge(smokingmp,hypertensionmp, on="Zip Code")
_ = pd.merge(_,obesitymp, on="Zip Code")
_ = pd.merge(_,sedenterymp, on="Zip Code")
_ = pd.merge(_,cholesterolmp, on="Zip Code")
_ = pd.merge(_,diabetesmp, on="Zip Code")
datameans = pd.merge(_,heartdmp, on="Zip Code")

display(datameans.head())
display(datameans.info())

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4792 entries, 0 to 4791
Data columns (total 8 columns):
Zip Code                    4792 non-null int64
Smoking Prevalence          4792 non-null float64
Hypertension Prevalence     4792 non-null float64
Obesity Prevalence          4792 non-null float64
Sedentarism Prevalence      4792 non-null float64
Cholesterol Prevalence      4792 non-null float64
Diabetes Prevalence         4792 non-null float64
Heart Disease Prevalence    4792 non-null float64
dtypes: float64(7), int64(1)
memory usage: 336.9 KB


None

In [119]:
#Open and import the fast food csv into a dataframe
dfff = pd.read_csv("fastfoodmaps_locations_2007.csv", header=None)
dfff.head()

,0,1,2,3,4,5,6,7,8,9
0,1,b,3601 N.W. 27th Avenue,Miami,FL,33142,(305) 638-3838,25.8092,-80.2400,0
1,2,b,8995 N. W. 7th Avenue,Miami,FL,33150,(305) 754-8453,25.8587,-80.2094,0
2,3,b,30390 South Dixie Highway,Homestead,FL,33030,(305) 247-7181,25.4849,-80.4610,0
3,4,b,7975 N. W. 27th Avenue,Miami,FL,33147,(305) 836-8152,25.8471,-80.2415,0
4,5,b,9201 South Dixie Highway,Miami,FL,33156,(305) 666-1130,25.6849,-80.3125,0


In [120]:
#Select and rename useful columns
dfff = dfff[[1,3,4,5,7,8]]
dfff.columns = ["Restaurant Count","City","State","Zip Code","Latitude","Longitude"]

# Map restaurant names to the Name column
dfff["Restaurant Count"] = 1
display(dfff.head())

display(dfff.info())

,Restaurant Count,City,State,Zip Code,Latitude,Longitude
0,1,Miami,FL,33142,25.8092,-80.2400
1,1,Miami,FL,33150,25.8587,-80.2094
2,1,Homestead,FL,33030,25.4849,-80.4610
3,1,Miami,FL,33147,25.8471,-80.2415
4,1,Miami,FL,33156,25.6849,-80.3125


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50002 entries, 0 to 50001
Data columns (total 6 columns):
Restaurant Count    50002 non-null int64
City                50002 non-null object
State               50002 non-null object
Zip Code            50002 non-null object
Latitude            50002 non-null float64
Longitude           50002 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 2.3+ MB


None

In [121]:
#Display zip code values
dfff["Zip Code"].sort_values()

36256         01001
26731         01007
37335         01008
37016         01008
37332         01008
37015         01008
37017         01011
35629         01013
19504         01020
31419         01020
49204         01020
39085         01020
26697    01020-3964
1987          01027
25669         01027
20898         01028
31740         01035
442           01035
24022         01035
47428         01035
14134         01035
28927         01040
20802         01040
49873         01040
30265         01040
33139         01040
1912          01040
37328         01056
2454          01056
32356         01056
            ...    
39760         99654
3023          99654
7439          99654
20167         99654
26279         99654
15848         99654
35796    99654-8105
32669         99669
41956         99669
48916         99687
15917         99701
43272         99701
12207         99701
23010         99701
31759    99701-1706
28184    99701-4092
3692          99702
3939          99703
40387         99705


In [122]:
# Remove trailing digits for zip codes
nohyphens = []
for x in dfff["Zip Code"]:
    nohyphens.append(x.replace('-',''))
dfff["Zip Code"] = nohyphens

zclist = []
for zc in dfff["Zip Code"]:
    if len(zc)>5:
        zclist.append(zc[:6])
    else:
        zclist.append(zc)
dfff["Zip Code"] = zclist
dfff["Zip Code"] = dfff["Zip Code"].astype('int')

In [123]:
# Group by zip code and get a count of restaurants
resgroups = dfff.groupby("Zip Code")["Restaurant Count"].sum().reset_index()
resgroups.head()

,Zip Code,Restaurant Count
0,1001,1
1,1007,1
2,1008,4
3,1011,1
4,1013,1


In [124]:
# Merge health dataframe with fast food location dataframe
datameansm = pd.merge(datameans,resgroups,how="left",on="Zip Code")

# Merge in populations by zip code
datameansm = pd.merge(datameansm,popzipgroups,on="Zip Code")

# Fill NaN values with 0
datameansm = datameansm.fillna(0)

display(datameansm.head())
display(datameansm.info())
display(datameansm.describe())

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,Population
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000,2.0,9138.149918
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,161714.819228
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,86339.551507
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,0.0,77817.567261
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035,2.0,173742.561568


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4783 entries, 0 to 4782
Data columns (total 10 columns):
Zip Code                    4783 non-null int64
Smoking Prevalence          4783 non-null float64
Hypertension Prevalence     4783 non-null float64
Obesity Prevalence          4783 non-null float64
Sedentarism Prevalence      4783 non-null float64
Cholesterol Prevalence      4783 non-null float64
Diabetes Prevalence         4783 non-null float64
Heart Disease Prevalence    4783 non-null float64
Restaurant Count            4783 non-null float64
Population                  4783 non-null float64
dtypes: float64(9), int64(1)
memory usage: 411.0 KB


None

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,Population
count,4783.000000,4783.000000,4783.000000,4783.000000,4783.000000,4783.000000,4783.000000,4783.000000,4783.000000,4783.000000
mean,59458.221200,18.077972,30.155647,30.089501,24.885293,34.373713,10.469872,5.754752,3.787372,147677.905052
std,29122.616001,5.565528,7.677159,7.633868,8.592858,4.997927,3.915990,1.914441,3.716725,124784.147436
min,1101.000000,2.470610,5.700000,10.519778,8.300000,8.500000,0.800000,0.600000,0.000000,10.642005
25%,33422.000000,14.022261,25.351937,24.561354,18.143860,31.867114,7.774196,4.521908,1.000000,42992.957440
50%,66103.000000,17.450789,29.182470,29.325324,23.596593,34.786413,9.781051,5.634234,3.000000,127362.531551
75%,85297.500000,21.607420,34.030853,34.867817,30.658107,37.499951,12.500000,6.887385,6.000000,215881.436811
max,99577.000000,40.900000,69.300000,56.000000,52.308167,52.100000,28.158736,21.000000,21.000000,797425.972598


In [125]:
# Round population values
datameansm.Population = datameansm.Population.astype("int")
datameansm.Population

0         9138
1       161714
2        86339
3        77817
4       173742
5       203013
6        97885
7        94881
8        18888
9        49816
10       61795
11      169680
12      140285
13      250605
14      184185
15      135129
16       62467
17       22291
18      142754
19      147673
20       30447
21      323937
22      170833
23      105960
24      213161
25      220728
26      187399
27      293069
28       11148
29      134741
         ...  
4753    241368
4754     94589
4755    166749
4756     75666
4757      7708
4758    135347
4759     49136
4760    323997
4761    143802
4762     48059
4763     98842
4764    139247
4765     83902
4766    267319
4767     33540
4768     54778
4769    253052
4770    239994
4771     14605
4772     20227
4773    129533
4774    140408
4775    103340
4776     71573
4777     10762
4778     18202
4779     17419
4780     17990
4781     59729
4782    185144
Name: Population, Length: 4783, dtype: int32

In [126]:
datacopy = datameansm

# Create a new column specifying population group by size
popgroups = []
for pop in datacopy["Population"]:
    if pop >80000:
        popgroups.append(">80000")
    if pop <20000:
        popgroups.append("<20000")
    if pop in range(20000,40000):
        popgroups.append("20000-40000")
    if pop in range(40000,60000):
        popgroups.append("40000-60000")
    if pop in range(60000,80000):
        popgroups.append("60000-80000")
        
datacopy["Population Group"]= popgroups
datacopy.head()

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,Population,Population Group
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000,2.0,9138,<20000
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,161714,>80000
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,86339,>80000
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,0.0,77817,60000-80000
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035,2.0,173742,>80000


In [127]:
# Create a new column specifying restaurant group by sum
restgroups = []
for count in datacopy["Restaurant Count"]:
    if count in range(0,8):
        restgroups.append("0-7")
    if count in range(8, 22):
        restgroups.append("8-21")
        
datacopy["Restaurant Group"]= restgroups
datacopy.head()

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,Population,Population Group,Restaurant Group
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000,2.0,9138,<20000,0-7
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,161714,>80000,0-7
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,86339,>80000,0-7
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,0.0,77817,60000-80000,0-7
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035,2.0,173742,>80000,0-7


In [128]:
# Import uszipcode to retrieve income for zip codes
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
zipstats = search.by_median_household_income(lower=-1, upper=2147483648, zipcode_type='Standard', 
                                            sort_by='median_household_income', ascending=False, returns=999999)

display(zipstats)

[SimpleZipcode(zipcode='81335', zipcode_type='Standard', major_city='Yellow Jacket', post_office_city='Yellow Jacket, CO', common_city_list=['Yellow Jacket'], county='Montezuma County', state='CO', lat=37.54, lng=-108.72, timezone='Mountain', radius_in_miles=2.0, area_code_list=['970'], population=131, population_density=5.0, land_area_in_sqmi=27.12, water_area_in_sqmi=0.0, housing_units=64, occupied_housing_units=54, median_home_value=301400, median_household_income=250001, bounds_west=-108.849361, bounds_east=-108.683032, bounds_north=37.573355, bounds_south=37.435026),
 SimpleZipcode(zipcode='21405', zipcode_type='Standard', major_city='Annapolis', post_office_city='Annapolis, MD', common_city_list=['Annapolis', 'Sherwood Forest', 'Sherwood Frst'], county='Anne Arundel County', state='MD', lat=39.03, lng=-76.55, timezone='Eastern', radius_in_miles=0.9659090909090909, area_code_list=['301', '410', '443'], population=544, population_density=549.0, land_area_in_sqmi=0.99, water_area_in

In [129]:
# Transform list of dictionaries to dataframe
list = []
for entry in zipstats:
    list.append(entry.to_dict())

zipstats_df = pd.DataFrame(list)
display(zipstats_df.head())

,area_code_list,bounds_east,bounds_north,bounds_south,bounds_west,common_city_list,county,housing_units,land_area_in_sqmi,lat,...,occupied_housing_units,population,population_density,post_office_city,radius_in_miles,state,timezone,water_area_in_sqmi,zipcode,zipcode_type
0,[970],-108.683032,37.573355,37.435026,-108.849361,[Yellow Jacket],Montezuma County,64,27.12,37.540,...,54,131,5.0,"Yellow Jacket, CO",2.000000,CO,Mountain,0.00,81335,Standard
1,"[301, 410, 443]",-76.529878,39.040506,39.017669,-76.570564,"[Annapolis, Sherwood Forest, Sherwood Frst]",Anne Arundel County,340,0.99,39.030,...,203,544,549.0,"Annapolis, MD",0.965909,MD,Eastern,0.75,21405,Standard
2,[503],-121.679473,45.366297,45.215218,-121.872530,"[Government Camp, Government Cp, Timberline Lo...",Clackamas County,713,38.01,45.300,...,114,217,6.0,"Government Camp, OR",2.000000,OR,Pacific,0.10,97028,Standard
3,"[281, 346, 713, 832]",-95.355302,29.757155,29.749765,-95.364051,[Houston],Harris County,471,0.12,29.754,...,249,366,2965.0,"Houston, TX",0.284091,TX,Central,0.00,77010,Standard
4,[307],-105.793154,41.128117,40.514119,-106.195438,"[Jelm, Laramie]",Albany County,170,384.84,41.060,...,54,100,0.0,"Jelm, WY",6.000000,WY,Mountain,0.85,82063,Standard


In [130]:
# Select zipcode and income columns
zipincome = zipstats_df[["zipcode","median_household_income"]]
zipincome.zipcode = zipincome.zipcode.astype("int")
display(zipincome.head())
display(zipincome.info())

,zipcode,median_household_income
0,81335,250001
1,21405,250001
2,97028,250001
3,77010,250001
4,82063,250001


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28847 entries, 0 to 28846
Data columns (total 2 columns):
zipcode                    28847 non-null int32
median_household_income    28847 non-null int64
dtypes: int32(1), int64(1)
memory usage: 338.1 KB


None

In [131]:
# Merge income data with dataframe
_ = pd.merge(datacopy,zipincome,how="left",left_on="Zip Code", right_on="zipcode")
_ = _.drop("zipcode", axis=1)
display(_.head())
display(_.info())
data = _

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,Population,Population Group,Restaurant Group,median_household_income
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000,2.0,9138,<20000,0-7,NaN
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,161714,>80000,0-7,32273.0
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,86339,>80000,0-7,18402.0
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,0.0,77817,60000-80000,0-7,21737.0
4,1108,23.844497,30.236068,35.381122,34.251449,35.500119,12.581968,6.943035,2.0,173742,>80000,0-7,34064.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4783 entries, 0 to 4782
Data columns (total 13 columns):
Zip Code                    4783 non-null int64
Smoking Prevalence          4783 non-null float64
Hypertension Prevalence     4783 non-null float64
Obesity Prevalence          4783 non-null float64
Sedentarism Prevalence      4783 non-null float64
Cholesterol Prevalence      4783 non-null float64
Diabetes Prevalence         4783 non-null float64
Heart Disease Prevalence    4783 non-null float64
Restaurant Count            4783 non-null float64
Population                  4783 non-null int32
Population Group            4783 non-null object
Restaurant Group            4783 non-null object
median_household_income     4395 non-null float64
dtypes: float64(9), int32(1), int64(1), object(2)
memory usage: 504.5+ KB


None

In [132]:
# Export to file
export_csv = data.to_csv('heart_disease_restaurants.csv', header=True)